In [2]:
import pickle 
import numpy as np
import pandas as pd
import datetime 
from functools import reduce
def convert_to_datetime(input):
    return datetime.datetime.strptime(input, "%Y-%m").date()
def merge_dataframes_on_column(dfs, column):
    if column == 'index':
        return reduce(lambda left,right: pd.merge(left,right,left_index=True,right_index=True), dfs)   
    else: 
        return reduce(lambda left,right: pd.merge(left,right,on=column), dfs)

In [3]:
with open('data/EIA_DATA.pkl', 'rb') as f:
    eia_data = pickle.load(f)

with open('data/FRED_DATA.pkl', 'rb') as f:
    fred_data = pickle.load(f)
eia_data = list(map(lambda df: df.rename(columns={'value': df['seriesId'][0]}).drop(columns=['seriesId']), eia_data))

In [4]:
eia_data = merge_dataframes_on_column(eia_data, 'period').set_index('period')
fred_data = merge_dataframes_on_column(fred_data, 'index')

In [5]:
df = eia_data.merge(fred_data, left_index=True, right_index=True)
df['forecast'] = df['WTISPLC'].shift(-1)

### Data Split

In [6]:
training_threshold = int(len(df) * 0.8)
validation_threshold = int(len(df) * 0.9)
testing_threshold = int(len(df))
training_data = df.iloc[:training_threshold]
validation_data = df.iloc[training_threshold:validation_threshold]
testing_data = df.iloc[validation_threshold:testing_threshold]

In [7]:
training_data

,PAPR_NONOPEC,PAPR_OPEC,PATC_OECD,PATC_NON_OECD,COPRPUS,CORIPUS,PASC_OECD_T3,COPS_OPEC,COPC_OPEC,T3_STCHANGE_OOECD,...,CPIENGSL,CAPG211S,CAPUTLG211S,IPG211S,IPG211111CN,INDPRO,IPN213111N,PCU211211,WTISPLC,forecast
2021-04-01,64.841425,29.160386,43.210662,52.698810,11.317989,15.162167,2911.576995,6.425,30.310,-0.272100,...,228.914,131.0718,94.3454,123.6602,165.4243,98.3317,85.6164,153.0,61.72,62.33
2021-03-01,64.768107,29.060352,43.506050,52.345726,11.351135,14.387129,2915.549210,6.425,30.320,1.950097,...,228.547,131.3459,93.1234,122.3138,165.8403,98.1929,81.6001,164.0,62.33,59.04
2021-02-01,61.576654,28.933585,41.886358,52.530509,9.915936,12.370929,2968.122920,6.435,30.220,1.249464,...,218.972,131.6582,81.8653,107.7823,150.8145,95.5072,80.7230,189.1,59.04,52.00
2021-01-01,64.471880,29.407937,41.774594,51.224158,11.137354,14.541839,3037.143340,5.525,29.730,-0.509581,...,209.996,132.0004,92.2854,121.8171,169.2416,98.8135,75.6223,140.8,52.00,47.02
2020-12-01,63.831982,29.248533,43.151512,51.102848,11.171507,14.139806,3027.590711,5.835,29.990,0.889581,...,205.228,132.3654,90.8968,120.3160,169.6635,98.2039,68.7129,129.3,47.02,40.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-02-01,53.102216,31.879552,51.636768,36.560326,5.117898,14.435143,2629.067000,2.410,29.878,0.230000,...,192.310,61.4995,91.7316,56.4145,78.6507,100.7166,148.4246,248.3,59.26,54.57
2007-01-01,52.761004,31.953318,49.907974,36.590908,5.107808,14.991581,2693.382000,2.190,29.605,-0.023613,...,190.281,61.4885,91.7345,56.4061,78.4959,99.7546,145.8005,217.7,54.57,62.03
2006-12-01,52.752568,32.183543,51.046333,35.388299,5.186066,15.353741,2688.890000,2.110,29.946,0.655839,...,192.700,61.4827,92.9875,57.1712,79.6982,100.1125,145.9131,256.2,62.03,59.37
2006-11-01,52.896737,32.288543,51.100165,35.949476,5.064209,15.008733,2734.947000,2.010,29.951,0.209267,...,184.100,61.4822,92.0694,56.6063,77.8246,99.0963,145.3853,244.5,59.37,58.88
